In [1]:
import kaggle

! kaggle competitions download -c tabular-playground-series-sep-2022

tabular-playground-series-sep-2022.zip: Skipping, found more recently modified local copy (use --force to force download)


# IMPORTING LIBRARIES AND DATA

In [171]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from xgboost import XGBRegressor,XGBClassifier
from sklearn import metrics as met

In [3]:
df_train=pd.read_csv("C:/Users/nagul/Desktop/Python Work/tabular-playground-series-sep-2022/train.csv")
df_test=pd.read_csv("C:/Users/nagul/Desktop/Python Work/tabular-playground-series-sep-2022/test.csv")

# SOME EXPLORATORY ANALYSIS

In [4]:
df_train.tail()

,row_id,date,country,store,product,num_sold
70123,70123,2020-12-31,Spain,KaggleMart,Kaggle for Kids: One Smart Goose,614
70124,70124,2020-12-31,Spain,KaggleRama,Kaggle Advanced Techniques,215
70125,70125,2020-12-31,Spain,KaggleRama,Kaggle Getting Started,158
70126,70126,2020-12-31,Spain,KaggleRama,Kaggle Recipe Book,135
70127,70127,2020-12-31,Spain,KaggleRama,Kaggle for Kids: One Smart Goose,202


In [5]:
print("Original training set has this many rows:", df_train.shape[0])
print("Original training set has this many columns:", df_train.shape[1])

Original training set has this many rows: 70128
Original training set has this many columns: 6


In [6]:
#SOME SUMMARY STATISTICS
df_train.describe(include="all")

,row_id,date,country,store,product,num_sold
count,70128.000000,70128,70128,70128,70128,70128.000000
unique,NaN,1461,6,2,4,NaN
top,NaN,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,NaN
freq,NaN,48,11688,35064,17532,NaN
mean,35063.500000,NaN,NaN,NaN,NaN,194.296986
std,20244.354176,NaN,NaN,NaN,NaN,126.893874
min,0.000000,NaN,NaN,NaN,NaN,19.000000
25%,17531.750000,NaN,NaN,NaN,NaN,95.000000
50%,35063.500000,NaN,NaN,NaN,NaN,148.000000
75%,52595.250000,NaN,NaN,NaN,NaN,283.000000


In [7]:
#CHECKING TO SEE IF THERE IS AMPLE NUMBER OF EACH CLASS
print(df_train["product"].value_counts())
print(df_train["country"].value_counts())
print(df_train["store"].value_counts())

Kaggle Advanced Techniques          17532
Kaggle Getting Started              17532
Kaggle Recipe Book                  17532
Kaggle for Kids: One Smart Goose    17532
Name: product, dtype: int64
Belgium    11688
France     11688
Germany    11688
Italy      11688
Poland     11688
Spain      11688
Name: country, dtype: int64
KaggleMart    35064
KaggleRama    35064
Name: store, dtype: int64


In [8]:
#CHECKING FOR NA'S

print(df_train.isna().sum())
print("\n",df_train.dtypes)

row_id      0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64

 row_id       int64
date        object
country     object
store       object
product     object
num_sold     int64
dtype: object


In [9]:
#THERE ARE NO NA'S
df_train[df_train.isna().any(1)]

,row_id,date,country,store,product,num_sold


In [10]:
#CHANGING DATE COLUMN TO DATE TYPE AND EXTRACTING THE DAY AND MONTH
df_train["date"] = pd.to_datetime(df_train["date"],format="%Y-%m-%d")
df_train.dtypes

row_id               int64
date        datetime64[ns]
country             object
store               object
product             object
num_sold             int64
dtype: object

In [11]:
df_train["dayofweek"]=df_train["date"].dt.day_name()
df_train["month"]=df_train["date"].dt.month_name()

df_train.head()

,row_id,date,country,store,product,num_sold,dayofweek,month
0,0,2017-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,663,Sunday,January
1,1,2017-01-01,Belgium,KaggleMart,Kaggle Getting Started,615,Sunday,January
2,2,2017-01-01,Belgium,KaggleMart,Kaggle Recipe Book,480,Sunday,January
3,3,2017-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710,Sunday,January
4,4,2017-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,240,Sunday,January


In [12]:
#TRAINING DATA DOES NOT NEED ROW_ID AND DATE ANYMORE
modified_training_df=df_train.drop(columns=["row_id","date"],axis=1).copy()
modified_training_df.head()

,country,store,product,num_sold,dayofweek,month
0,Belgium,KaggleMart,Kaggle Advanced Techniques,663,Sunday,January
1,Belgium,KaggleMart,Kaggle Getting Started,615,Sunday,January
2,Belgium,KaggleMart,Kaggle Recipe Book,480,Sunday,January
3,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,710,Sunday,January
4,Belgium,KaggleRama,Kaggle Advanced Techniques,240,Sunday,January


In [13]:
#GETTING DUMMY VARIABLES AND DROPPING THE LABEL

X_encoded=pd.get_dummies(modified_training_df,
                         columns=["country","store","product",
                                  "dayofweek","month"])
y=X_encoded["num_sold"]
X_encoded=X_encoded.drop(columns=["num_sold"],axis=1)
X_encoded.head()

,country_Belgium,country_France,country_Germany,country_Italy,country_Poland,country_Spain,store_KaggleMart,store_KaggleRama,product_Kaggle Advanced Techniques,product_Kaggle Getting Started,...,month_December,month_February,month_January,month_July,month_June,month_March,month_May,month_November,month_October,month_September
0,1,0,0,0,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,1,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,1,1,0,...,0,0,1,0,0,0,0,0,0,0


In [14]:
#ensuring for categories that we have a reasonable number of each (0 or 1)
for columns in X_encoded:
    #print(X_encoded.columns.value_counts())
    print(X_encoded[columns].value_counts())

0    58440
1    11688
Name: country_Belgium, dtype: int64
0    58440
1    11688
Name: country_France, dtype: int64
0    58440
1    11688
Name: country_Germany, dtype: int64
0    58440
1    11688
Name: country_Italy, dtype: int64
0    58440
1    11688
Name: country_Poland, dtype: int64
0    58440
1    11688
Name: country_Spain, dtype: int64
1    35064
0    35064
Name: store_KaggleMart, dtype: int64
0    35064
1    35064
Name: store_KaggleRama, dtype: int64
0    52596
1    17532
Name: product_Kaggle Advanced Techniques, dtype: int64
0    52596
1    17532
Name: product_Kaggle Getting Started, dtype: int64
0    52596
1    17532
Name: product_Kaggle Recipe Book, dtype: int64
0    52596
1    17532
Name: product_Kaggle for Kids: One Smart Goose, dtype: int64
0    60144
1     9984
Name: dayofweek_Friday, dtype: int64
0    60096
1    10032
Name: dayofweek_Monday, dtype: int64
0    60144
1     9984
Name: dayofweek_Saturday, dtype: int64
0    60096
1    10032
Name: dayofweek_Sunday, dtype: int64


In [150]:
#no need to stratify, target is a value and not categories so it wont work
X_train,X_test,y_train,y_test=train_test_split(X_encoded,y,random_state=42,test_size=0.2)

In [151]:
#just peeking
X_train

,country_Belgium,country_France,country_Germany,country_Italy,country_Poland,country_Spain,store_KaggleMart,store_KaggleRama,product_Kaggle Advanced Techniques,product_Kaggle Getting Started,...,month_December,month_February,month_January,month_July,month_June,month_March,month_May,month_November,month_October,month_September
38747,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
66323,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
61459,0,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
50777,0,0,0,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,1,0,0
52928,0,0,0,0,1,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37194,0,0,0,0,0,1,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6265,0,0,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0
54886,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
860,0,0,0,0,0,1,0,1,1,0,...,0,0,1,0,0,0,0,0,0,0


In [179]:
#CUSTOM SCORING FUNCTION

def smape(A,F):
    return 100/len(A) * np.sum(2*np.abs(F-A)/(np.abs(A)+np.abs(F)))

smape_score=met.make_scorer(smape,greater_is_better=False)

In [230]:
#CREAING XGBOOST SHELL

xgbr = XGBRegressor(verbosity=0,seed=42,early_stopping_rounds=10,eval_metric=smape)
print(xgbr)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=10,
             enable_categorical=False,
             eval_metric=<function smape at 0x000001D9F1194310>, gamma=None,
             gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None,
             reg_alpha=None, reg_lambda=None, ...)


In [231]:
#FITTING THE MODEL WITH RESPECT TO PERFORMANCE ON EVALUATION SET

xgbr.fit(X_train,y_train,verbose=False,eval_set=[(X_test, y_test)])

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=10, enable_categorical=False,
             eval_metric=<function smape at 0x000001D9F1194310>, gamma=0,
             gpu_id=-1, grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=6, max_leaves=0,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=42, reg_alpha=0, reg_lambda=1, ...)

In [232]:
training_predictions=xgbr.predict(X_test)
smape(y_test,training_predictions)

15.735650089048507

# CROSS VALIDATION YIELDS SIMILAR RESULTS

In [246]:
cv_score=cross_val_score(xgbr,X_train,y_train,verbose=0,fit_params={'eval_set':[(X_test, y_test)]},cv=3,scoring=smape_score)

[0]	validation_0-rmse:165.63476	validation_0-smape:103.41786
[1]	validation_0-rmse:121.81730	validation_0-smape:60.62241
[2]	validation_0-rmse:92.73525	validation_0-smape:38.16904
[3]	validation_0-rmse:74.03970	validation_0-smape:27.03409
[4]	validation_0-rmse:62.43599	validation_0-smape:21.77087
[5]	validation_0-rmse:55.76773	validation_0-smape:19.34276
[6]	validation_0-rmse:51.90651	validation_0-smape:18.33085
[7]	validation_0-rmse:49.74744	validation_0-smape:17.88228
[8]	validation_0-rmse:48.02911	validation_0-smape:17.60739
[9]	validation_0-rmse:47.37106	validation_0-smape:17.58542
[10]	validation_0-rmse:46.90555	validation_0-smape:17.56207
[11]	validation_0-rmse:46.54235	validation_0-smape:17.39196
[12]	validation_0-rmse:46.35925	validation_0-smape:17.34136
[13]	validation_0-rmse:45.41422	validation_0-smape:16.94334
[14]	validation_0-rmse:45.06990	validation_0-smape:16.81983
[15]	validation_0-rmse:44.80002	validation_0-smape:16.75147
[16]	validation_0-rmse:44.68621	validation_0-sm

[40]	validation_0-rmse:43.32593	validation_0-smape:15.84695
[41]	validation_0-rmse:43.33651	validation_0-smape:15.84844
[42]	validation_0-rmse:43.33911	validation_0-smape:15.85488
[43]	validation_0-rmse:43.34460	validation_0-smape:15.85667
[44]	validation_0-rmse:43.34105	validation_0-smape:15.85383
[45]	validation_0-rmse:43.35632	validation_0-smape:15.85795
[46]	validation_0-rmse:43.36448	validation_0-smape:15.85703
[47]	validation_0-rmse:43.36760	validation_0-smape:15.85487
[48]	validation_0-rmse:43.37312	validation_0-smape:15.84772


# USING GRIDSEARCH TO FIND GOOD PARAMETERS

In [251]:
from sklearn.model_selection import GridSearchCV

#josh suggest using subsampling to prevent overfitting
#by using random subsets of the data (90%) and only random subset
#of columns (50%) per tree. 3-fold validation for speed.


#first bit of tuning
'''
param_grid = {
        'max_depth' : [2,3,4],
        'learning_rate' : [0.01,0.1,0.2],
        'gamma' : [0,0.25,1.0],
        #'reg_lambda': [0, 1.0, 10.0],
        #'scale_pos_weight' : [1,1.5,2],
        'seed':[42],
        'subsample':[0.8],
        'colsample_bytree':[0.5],
}
'''
#{'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.2, 'max_depth': 4, 'seed': 42, 'subsample': 0.8}
'''
param_grid = {
        'max_depth' : [4,5,6],
        'learning_rate' : [0.2,0.25,0.3],
        'gamma' : [0,0.1],
        #'reg_lambda': [0, 1.0, 10.0],
        #'scale_pos_weight' : [1,1.5,2],
        'seed':[42],
        'subsample':[0.8],
        'colsample_bytree':[0.5],
}
'''
#{'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.2, 'max_depth': 5, 'seed': 42, 'subsample': 0.8}
'''
param_grid = {
        'max_depth' : [5],
        'learning_rate' : [0.2],
        'gamma' : [0],
        'reg_lambda': [0, 1.0, 10.0],
        #scale_pos_weight' : [1,1.5,2],
        'seed':[42],
        'subsample':[0.8],
        'colsample_bytree':[0.5],
}
'''
#{'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.2, 'max_depth': 5, 'reg_lambda': 1.0, 
#'scale_pos_weight': 1, 'seed': 42, 'subsample': 0.8}

param_grid = {
        'max_depth' : [5],
        'learning_rate' : [0.15,0.2,0.25],
        'gamma' : [0],
        'reg_lambda': [1.0,2],
        'seed':[42],
        'subsample':[0.8],
        'colsample_bytree':[0.5],
}



#josh suggest using subsampling to prevent overfitting
#by using random subsets of the data (90%) and only random subset
#of columns (50%) per tree. 3-fold validation for speed.

optional_params = GridSearchCV(
    estimator=xgbr,
    param_grid=param_grid,
    scoring=smape_score,
    verbose=2,
    n_jobs=10,
    cv=3
)

optional_params.fit(X_train,y_train,eval_set=[(X_test, y_test)],verbose=False)
print("The best parameters are: \n",optional_params.best_params_)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
The best parameters are: 
 {'colsample_bytree': 0.5, 'gamma': 0, 'learning_rate': 0.2, 'max_depth': 5, 'reg_lambda': 1.0, 'seed': 42, 'subsample': 0.8}


In [34]:
xgbr = XGBRegressor(verbosity=0,seed=42,early_stopping_rounds=10,eval_metric=smape,colsample_bytree=0.5,learning_rate=0.2,
                   max_depth=5,reg_lambda=1,)
print(xgbr)

# NOW WE CAN EXTRACT THE BEST MODEL AND ASSESS PERFORMANCE ON TRAINING AND VALIDATION

In [252]:
optimizedxgb=optional_params.best_estimator_
optimizedxgb

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             early_stopping_rounds=10, enable_categorical=False,
             eval_metric=<function smape at 0x000001D9F1194310>, gamma=0,
             gpu_id=-1, grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.2, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=5, max_leaves=0,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, predictor='auto',
             random_state=42, reg_alpha=0, reg_lambda=1.0, ...)

In [253]:
#seeing sMAPE on validationset. Note that by test in this cell we actually meant validation set, not the submission set

predictions=optimizedxgb.predict(X_test)
print(smape(y_test,predictions))

15.675540912668954


In [254]:
#seeing sMAPE on training set

predictions=optimizedxgb.predict(X_train)
print(smape(y_train,predictions))

15.486921646105566


# NOW TAKE IN THE ACTUAL TEST DF, FORMAT IT LIKE THE TRAINING DATA AND APPLY MODEL

In [266]:
df_test["date"] = pd.to_datetime(df_test["date"],format="%Y-%m-%d")



df_test["dayofweek"]=df_test["date"].dt.day_name()
df_test["month"]=df_test["date"].dt.month_name()


row_id_values=df_test["row_id"]
modified_testing_df=df_test.drop(columns=["row_id","date"],axis=1).copy()


X_testing_encoded=pd.get_dummies(modified_testing_df,
                         columns=["country","store","product",
                                  "dayofweek","month"])


In [267]:
X_testing_encoded.shape

(17520, 31)

In [271]:
df_test

,row_id,date,country,store,product,dayofweek,month
0,70128,2021-01-01,Belgium,KaggleMart,Kaggle Advanced Techniques,Friday,January
1,70129,2021-01-01,Belgium,KaggleMart,Kaggle Getting Started,Friday,January
2,70130,2021-01-01,Belgium,KaggleMart,Kaggle Recipe Book,Friday,January
3,70131,2021-01-01,Belgium,KaggleMart,Kaggle for Kids: One Smart Goose,Friday,January
4,70132,2021-01-01,Belgium,KaggleRama,Kaggle Advanced Techniques,Friday,January
...,...,...,...,...,...,...,...
17515,87643,2021-12-31,Spain,KaggleMart,Kaggle for Kids: One Smart Goose,Friday,December
17516,87644,2021-12-31,Spain,KaggleRama,Kaggle Advanced Techniques,Friday,December
17517,87645,2021-12-31,Spain,KaggleRama,Kaggle Getting Started,Friday,December
17518,87646,2021-12-31,Spain,KaggleRama,Kaggle Recipe Book,Friday,December


In [268]:
#MAKING SURE DUMMY VARIABLES ALIGN

X_encoded, X_testing_encoded = X_encoded.align(X_testing_encoded, join ='inner', axis = 1)
X_testing_encoded.shape

(17520, 31)

In [270]:
#CREATING SUBMISSION DF
submission_predictions=optimizedxgb.predict(X_testing_encoded)
submission_df=pd.DataFrame({'row_id':row_id_values,'num_sold':submission_predictions})
submission_df

,row_id,num_sold
0,70128,437.639221
1,70129,331.467377
2,70130,279.462311
3,70131,455.111786
4,70132,149.185364
...,...,...
17515,87643,352.793945
17516,87644,100.383339
17517,87645,93.801147
17518,87646,75.060486


In [274]:
#WRITING SUBMISSION DF TO CSV FILE FOR SUBMISSION
submission_df.to_csv('submission-TabularPlaygroundSeries-Sep2022',index=False)

In [ ]:
#NEXT, TRY OTHER MODELS